<a href="https://colab.research.google.com/github/mimahb/spam-email-classifier/blob/main/spamemailclassifier2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
#load dataset
spam_df = pd.read_csv('/content/spam data.csv')

#convert to dataframe
spam_df = pd.DataFrame(spam_df, columns=['text', 'label'])

#shuffle data
spam_df = spam_df.sample(frac=1).reset_index(drop=True)

#stopwords common words to remove
stop_words = set(stopwords.words('english'))

#preprocess texts
def clean_text(text):
  if not isinstance(text, str):  # Convert non-strings to empty string
        text = ""
  words = word_tokenize(text.lower())  # Tokenize & lowercase
  return [word for word in words if word.isalpha() and word not in stop_words]


# Apply cleaning
spam_df["processed_text"] = spam_df["text"].apply(clean_text)

In [ ]:
from nltk import FreqDist
# Get most common words
all_words = [word for words in spam_df["processed_text"] for word in words]
word_features = list(FreqDist(all_words).keys())[:2000]

# Check if we got words
print("All Words Sample:", all_words[:10])  # Should print some words


# Function to convert text into features
def document_features(words):
    words = set(words)
    return {word: (word in words) for word in word_features}

# Convert dataset into feature sets
featuresets = [(document_features(words), label) for words, label in zip(spam_df["processed_text"], spam_df["label"])]

All Words Sample: ['using', 'tomcat', 'need', 'need', 'support', 'web', 'services', 'security', 'get', 'stuff']


In [ ]:
#train the naivebayes model
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy
from sklearn.model_selection import train_test_split

# Split dataset (80% train, 20% test)
train_set, test_set = train_test_split(featuresets, test_size=0.2, random_state=42)

# Train model
classifier = NaiveBayesClassifier.train(train_set)

# Test model
accuracy_score = accuracy(classifier, test_set)
print(f"Accuracy: {accuracy_score * 100:.2f}%")


Accuracy: 88.54%


In [ ]:
print("First 10 Word Features:", word_features[:10])

First 10 Word Features: ['author', 'kai', 'date', 'escapenumber', 'mon', 'jun', 'new', 'revision', 'websvn', 'http']


In [ ]:
#sample emails
emails = ['michael pobega wrote i\'m not sure if it\'s the mpl or mozilla that didn\'t allow the distribution of their images or the patching of programs without their knowledge but i think that is not dfsg free last time i looked the mozilla images were in an other licenses directory so not under the mpl and not licensed to others at all hope that helps mjr slef my opinion only see http people debian org mjr please follow']
processed_email = clean_text(emails[0])
email_features = document_features(processed_email)

make_predictions = classifier.classify(email_features)
print(make_predictions)

if make_predictions == 1:
  print("spam")
else:
  print('ham')


0
ham


In [ ]:
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Create and fit TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=2000)  # You can adjust max_features
vectorizer.fit(spam_df["text"])  # Fit to your text data

# Save the vectorizer
joblib.dump(vectorizer, 'vectorizer.pkl')

joblib.dump(classifier, 'spam_classifier.pkl')



['spam_classifier.pkl']

In [ ]:
!pip install fastapi nest-asyncio pyngrok uvicorn


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import joblib

# Apply nested event loop patch for Colab
nest_asyncio.apply()

# Load the trained model and vectorizer
model = joblib.load("spam_classifier.pkl")
vectorizer = joblib.load("vectorizer.pkl")

app = FastAPI()

class EmailText(BaseModel):
    text: str

@app.post("/predict")
def predict_spam(data: EmailText):
    transformed_text = vectorizer.transform([data.text])
    prediction = model.predict(transformed_text)
    label = "spam" if prediction[1] == 0 else "not spam"
    return {"prediction": label}

from threading import Thread
!ngrok authtoken 2vMgmtnlQzUPNRkWRxM5hXwjdKV_546Deti2Ad6Fm8tU9ZQz9

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

t = Thread(target=run)
t.start()

!pip install pyngrok
from pyngrok import ngrok

# Set up a tunnel to the FastAPI server
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


Exception in thread Thread-10 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-20-d4f66e598a1e>", line 30, in run
NameError: name 'uvicorn' is not defined


Public URL: NgrokTunnel: "https://46cf-146-148-107-171.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
!pip install streamlit

import streamlit as st
import requests

with open('app.py', 'w') as f:
  f.write("""
st.title('Spam Email Classification')
st.write("This is a sample Streamlit app for spam email classification!")

st.title("📧 Spam Email Classifier")
email = st.text_area("Enter email content:")

if st.button("Classify"):
    response = requests.post(API_URL, json={"text": email})
    prediction = response.json()["prediction"]
    st.success(f"The email is classified as: **{prediction.upper()}**")""")

 #Expose Streamlit app using ngrok
from pyngrok import ngrok

# Set up a tunnel to the Streamlit app (port 8501 is the default Streamlit port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")


API_URL = "http://localhost:8000/"

!streamlit run app.py &  # Run Streamlit in the background

Streamlit app is live at: NgrokTunnel: "https://235f-146-148-107-171.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://146.148.107.171:8501

